In [4]:
!nvidia-smi
!nvcc --version

Tue Sep 23 21:38:15 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   36C    P8             11W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [5]:
%%writefile gemm.cu
#include <cuda_runtime.h>
#include <cstdio>
#include <cstdlib>
#include <vector>
#include <random>
#include <cstring>
#include <cmath>
#include <string>
#include <iostream>

#ifndef CHECK_CUDA
#define CHECK_CUDA(call) do {                                      \
  cudaError_t _e = (call);                                         \
  if (_e != cudaSuccess) {                                         \
    fprintf(stderr, "CUDA error %s:%d: %s\n", __FILE__, __LINE__,  \
            cudaGetErrorString(_e));                               \
    std::exit(1);                                                  \
  }                                                                \
} while (0)
#endif

// Row-major indexing helpers
__host__ __device__ inline int idxA(int r, int c, int K) { return r * K + c; }
__host__ __device__ inline int idxB(int r, int c, int N) { return r * N + c; }
__host__ __device__ inline int idxC(int r, int c, int N) { return r * N + c; }
__host__ __device__ inline int idxD(int r, int c, int N) { return r * N + c; }

// ------------------------- NAIVE KERNEL -------------------------
__global__ void gemm_naive_kernel(const float* __restrict__ A,
                                  const float* __restrict__ B,
                                  const float* __restrict__ C,
                                  float* __restrict__ D,
                                  int M, int N, int K,
                                  float alpha, float beta) {
  int row = blockIdx.y * blockDim.y + threadIdx.y; // [0, M)
  int col = blockIdx.x * blockDim.x + threadIdx.x; // [0, N)

  if (row >= M || col >= N) return;

  float acc = 0.0f;
  for (int k = 0; k < K; ++k) {
    acc = fmaf(A[idxA(row, k, K)], B[idxB(k, col, N)], acc); // A[row,k]*B[k,col] + acc
  }
  D[idxD(row, col, N)] = alpha * acc + beta * C[idxC(row, col, N)];
}

// --------------------- TILED/SHARED-MEM KERNEL ------------------
// Each thread computes 1 output element. Tiles are BLOCK x BLOCK.
// We pad shared tiles by +1 on the second dim to reduce bank conflicts.
template<int BLOCK>
__global__ void gemm_tiled_kernel(const float* __restrict__ A,
                                  const float* __restrict__ B,
                                  const float* __restrict__ C,
                                  float* __restrict__ D,
                                  int M, int N, int K,
                                  float alpha, float beta) {
  __shared__ float As[BLOCK][BLOCK + 1];
  __shared__ float Bs[BLOCK][BLOCK + 1];

  int row = blockIdx.y * BLOCK + threadIdx.y;
  int col = blockIdx.x * BLOCK + threadIdx.x;

  float acc = 0.0f;
  const int tiles = (K + BLOCK - 1) / BLOCK;

  for (int t = 0; t < tiles; ++t) {
    int aCol = t * BLOCK + threadIdx.x;
    int bRow = t * BLOCK + threadIdx.y;

    // Load tiles into shared memory (guarded)
    As[threadIdx.y][threadIdx.x] =
        (row < M && aCol < K) ? A[idxA(row, aCol, K)] : 0.0f;
    Bs[threadIdx.y][threadIdx.x] =
        (bRow < K && col < N) ? B[idxB(bRow, col, N)] : 0.0f;

    __syncthreads();

    #pragma unroll
    for (int k = 0; k < BLOCK; ++k) {
      acc = fmaf(As[threadIdx.y][k], Bs[k][threadIdx.x], acc);
    }
    __syncthreads();
  }

  if (row < M && col < N) {
    D[idxD(row, col, N)] = alpha * acc + beta * C[idxC(row, col, N)];
  }
}

// --------------------------- CPU REF ----------------------------
void cpu_gemm(const std::vector<float>& A,
              const std::vector<float>& B,
              const std::vector<float>& C,
              std::vector<float>& D,
              int M, int N, int K,
              float alpha, float beta) {
  for (int i = 0; i < M; ++i) {
    for (int j = 0; j < N; ++j) {
      double acc = 0.0;
      for (int k = 0; k < K; ++k) {
        acc += static_cast<double>(A[idxA(i,k,K)]) * static_cast<double>(B[idxB(k,j,N)]);
      }
      D[idxD(i,j,N)] = static_cast<float>(alpha * acc + beta * C[idxC(i,j,N)]);
    }
  }
}

// --------------------------- UTILS ------------------------------
float max_abs_diff(const std::vector<float>& X, const std::vector<float>& Y) {
  float m = 0.0f;
  for (size_t i = 0; i < X.size(); ++i) {
    m = fmaxf(m, fabsf(X[i] - Y[i]));
  }
  return m;
}

double gflops_gemm(long long M, long long N, long long K, double ms) {
  // GEMM does ~2*M*N*K MACs (+ 2*M*N for axpby on C), count all as FLOPs
  double flops = 2.0 * M * N * K + 2.0 * M * N;
  return flops / (ms * 1e6); // GFLOP/s
}

struct Args {
  int M=1024, N=1024, K=1024;
  int repeat=20;
  float alpha=1.0f, beta=1.0f;
  std::string kernel="tiled"; // "naive" or "tiled"
  int block=32;               // tile size for tiled kernel
  int seed=42;
};

Args parse_args(int argc, char** argv) {
  Args a;
  for (int i=1; i<argc; ++i) {
    std::string s(argv[i]);
    auto next = [&](int &i){ return std::string(argv[++i]); };
    if (s=="--m") a.M = std::stoi(next(i));
    else if (s=="--n") a.N = std::stoi(next(i));
    else if (s=="--k") a.K = std::stoi(next(i));
    else if (s=="--alpha") a.alpha = std::stof(next(i));
    else if (s=="--beta") a.beta = std::stof(next(i));
    else if (s=="--repeat") a.repeat = std::stoi(next(i));
    else if (s=="--kernel") a.kernel = next(i);
    else if (s=="--block") a.block = std::stoi(next(i));
    else if (s=="--seed") a.seed = std::stoi(next(i));
    else if (s=="-h" || s=="--help") {
      printf("Usage: ./gemm [--m M] [--n N] [--k K] [--alpha A] [--beta B] "
             "[--repeat R] [--kernel naive|tiled] [--block 16|32] [--seed S]\n");
      std::exit(0);
    }
  }
  return a;
}

int main(int argc, char** argv) {
  Args args = parse_args(argc, argv);
  int M = args.M, N = args.N, K = args.K;
  float alpha = args.alpha, beta = args.beta;

  // Host buffers
  std::vector<float> A(M*K), B(K*N), C(M*N), D(M*N), D_ref(M*N);

  // Init random
  std::mt19937 rng(args.seed);
  std::uniform_real_distribution<float> dist(-1.0f, 1.0f);
  for (auto &x : A) x = dist(rng);
  for (auto &x : B) x = dist(rng);
  for (auto &x : C) x = dist(rng);

  // Device buffers
  float *dA=nullptr, *dB=nullptr, *dC=nullptr, *dD=nullptr;
  CHECK_CUDA(cudaMalloc(&dA, sizeof(float)*A.size()));
  CHECK_CUDA(cudaMalloc(&dB, sizeof(float)*B.size()));
  CHECK_CUDA(cudaMalloc(&dC, sizeof(float)*C.size()));
  CHECK_CUDA(cudaMalloc(&dD, sizeof(float)*D.size()));
  CHECK_CUDA(cudaMemcpy(dA, A.data(), sizeof(float)*A.size(), cudaMemcpyHostToDevice));
  CHECK_CUDA(cudaMemcpy(dB, B.data(), sizeof(float)*B.size(), cudaMemcpyHostToDevice));
  CHECK_CUDA(cudaMemcpy(dC, C.data(), sizeof(float)*C.size(), cudaMemcpyHostToDevice));

  // Launch params
  dim3 block(16,16);
  dim3 grid((N + block.x - 1)/block.x, (M + block.y - 1)/block.y);

  // Warmup + timing
  cudaEvent_t start, stop;
  CHECK_CUDA(cudaEventCreate(&start));
  CHECK_CUDA(cudaEventCreate(&stop));

  // Select kernel
  auto run_naive = [&](){
    gemm_naive_kernel<<<grid, block>>>(dA, dB, dC, dD, M, N, K, alpha, beta);
  };

  auto run_tiled = [&](){
    if (args.block == 16) {
      dim3 b(16,16), g((N+15)/16,(M+15)/16);
      gemm_tiled_kernel<16><<<g, b>>>(dA,dB,dC,dD,M,N,K,alpha,beta);
    } else { // default 32
      dim3 b(32,32), g((N+31)/32,(M+31)/32);
      gemm_tiled_kernel<32><<<g, b>>>(dA,dB,dC,dD,M,N,K,alpha,beta);
    }
  };

  // Warmup
  if (args.kernel == "naive") run_naive(); else run_tiled();
  CHECK_CUDA(cudaDeviceSynchronize());

  // Time
  CHECK_CUDA(cudaEventRecord(start));
  for (int i=0;i<args.repeat;++i) {
    if (args.kernel == "naive") run_naive(); else run_tiled();
  }
  CHECK_CUDA(cudaEventRecord(stop));
  CHECK_CUDA(cudaEventSynchronize(stop));
  float ms=0.0f;
  CHECK_CUDA(cudaEventElapsedTime(&ms, start, stop));
  ms /= args.repeat;

  // Copy back
  CHECK_CUDA(cudaMemcpy(D.data(), dD, sizeof(float)*D.size(), cudaMemcpyDeviceToHost));

  // CPU reference (smaller shapes are fast; for big shapes this is still okay but slower)
  cpu_gemm(A,B,C,D_ref,M,N,K,alpha,beta);

  // Check max abs diff
  float mad = max_abs_diff(D, D_ref);

  // Report
  double gflops = gflops_gemm(M,N,K, ms);
  std::cout << "Kernel = " << args.kernel
            << "  M=" << M << " N=" << N << " K=" << K
            << "  alpha=" << alpha << " beta=" << beta << "\n";
  std::cout << "Avg time: " << ms << " ms   Throughput: "
            << gflops << " GFLOP/s\n";
  std::cout << "Max |GPU-CPU|: " << mad << "\n";

  // Cleanup
  CHECK_CUDA(cudaFree(dA));
  CHECK_CUDA(cudaFree(dB));
  CHECK_CUDA(cudaFree(dC));
  CHECK_CUDA(cudaFree(dD));
  CHECK_CUDA(cudaEventDestroy(start));
  CHECK_CUDA(cudaEventDestroy(stop));
  return 0;
}


Overwriting gemm.cu


In [6]:
!nvcc -O3 -std=c++17 gemm.cu -o gemm

In [7]:
# Naïve kernel (correct but slower)
!./gemm --m 512 --n 512 --k 512 --alpha 1.25 --beta 0.75 --kernel naive --repeat 50


Kernel = naive  M=512 N=512 K=512  alpha=1.25 beta=0.75
Avg time: 0.0002752 ms   Throughput: 977325 GFLOP/s
Max |GPU-CPU|: 43.7532


In [8]:
# Tiled/shared-memory kernel (faster)
!./gemm --m 1024 --n 1024 --k 1024 --alpha 1.0 --beta 1.0 --kernel tiled --block 32 --repeat 30


Kernel = tiled  M=1024 N=1024 K=1024  alpha=1 beta=1
Avg time: 0.000349867 ms   Throughput: 6.144e+06 GFLOP/s
Max |GPU-CPU|: 51.3591
